In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#como el navegador a usar es chrome, hemos descargado ya el driver de la página, guardamos la ruta en una variable
#chrome_path = 'chromedriver_linux64/chromedriver'
#driver = webdriver.Chrome(executable_path = chrome_path)

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

In [16]:
eyecream_url = []

In [17]:
for i in range (1,33):
    url = f'https://www.stylevana.com/en_US/skincare/eye-lip-care.html?p={i}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    eyecream_list = soup.find_all('div', class_='product-item-info')
    for item in eyecream_list:
        for link in item.find_all('a', class_='product-image', href=True): 
            eyecream_url.append(link['href'])
print(len(eyecream_url))

382


with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

In [18]:
len(eyecream_url)

382

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [21]:
eyecream_url[382]

IndexError: list index out of range

In [22]:
eye_creams = []

In [25]:
for link in eyecream_url[345:382]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        name = soup.find('h1', class_='product-name-h1').text
        details = (soup.find('div', id='product-view-details').text.strip()) # details
        ingredients = soup.find_all('div', id="product-view-ingredients") #ingredients
        for item in ingredients:
            ingredients = item.find('div', class_='panel-body').text.strip()
    except:
        name = 'no info'
        details = 'no info'
        ingredients = 'no info'
        
    eyecream_info = {
    'name': name,
    'details': details,
    'ingredients': ingredients}
    
    eye_creams.append(eyecream_info)

In [26]:
print(len(eye_creams))

382


## 3. PASARLO A DATAFRAME

In [28]:
eyecreams_df = pd.DataFrame(eye_creams)
eyecreams_df

,name,details,ingredients
0,LANEIGE - Lip Sleeping Mask,Product Type: Lip Mask\n\n\nProduct Unit: 20g\...,"Diisostearyl Malate, Hydrogenated Polyisobuten..."
1,PURITO - Centella Green Level Eye Cream - 30ml,Cruelty Free: Yes\n\n\nProduct Type: Eye Cream...,"Centella Asiatica Extract, Water, Hydrogenated..."
2,iUNIK - Propolis Vitamin Eye Cream - 30ml,"Skin Concern: Anti-aging/ Wrinkles, Dark Circl...",[]
3,Benton - Fermentation Eye Cream,Cruelty Free: Yes\n\n\nProduct Type: Eye Cream...,"Galactomyces Ferment Filtrate, Aqua (Water), B..."
4,Etude House - My Lash Serum,Product Type: Eyelash Essence\n\n\nSkin Type: ...,"Water, Butylene Glycol, Alcohol, Glycerin, Alo..."
...,...,...,...
377,Mongdies - Lip Treatment - 15g,Product Type: Lip Mask\n\n\nSkin Concern: Irri...,"Caprylic/Capric Triglyceride, Polybutene, Bis-..."
378,IOPE - Lip Nutri-Volume Serum Pure - 15g,"Skin Concern: Anti-aging/ Wrinkles, Dryness/ H...",[]
379,LITS - LITS Revival - Line Zero - 12g,Product Type: Eye Cream\n\n\nSkin Concern: Ant...,"Water, Dimethicone, Cyclopentasiloxane, Silica..."
380,ISOI - Bulgarian Intensive Age-Control Eye Cre...,Product Type: Eye Cream\n\n\nSkin Concern: Ant...,"Water, Glycerin, Octyldodecyl Myristate, Cetyl..."


In [29]:
eyecreams = eyecreams_df.to_csv('data/eyecreams.csv')